# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherpy_results = pd.read_csv("output_data/cities.csv")
weatherpy_results

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dubai,25.2582,55.3047,95.29,12,0,4.61,AE,1648189073
1,1,puerto ayora,-0.7393,-90.3518,77.86,82,85,6.22,EC,1648189257
2,2,saint-philippe,-21.3585,55.7679,85.59,75,17,8.01,RE,1648189258
3,3,tuktoyaktuk,69.4541,-133.0374,-14.82,94,5,11.59,CA,1648189258
4,4,chern,53.4523,36.9145,33.04,96,100,3.74,RU,1648189259
...,...,...,...,...,...,...,...,...,...,...
545,545,haines junction,60.7522,-137.5108,30.15,69,100,1.14,CA,1648189690
546,546,toamasina,-18.1667,49.3833,86.00,74,40,3.44,MG,1648189691
547,547,valparaiso,-33.0393,-71.6273,61.48,60,0,4.61,CL,1648189630
548,548,sobolevo,54.4299,31.8996,30.56,100,90,3.44,RU,1648189692


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weatherpy_results[["Lat", "Lng"]].astype(float)
weight = weatherpy_results["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.

#Wind speed less than 10 mph.

#Zero cloudiness.

weather_criteria_df = weatherpy_results.loc[(weatherpy_results["Max Temp"] <= 80) & 
                                            (weatherpy_results["Max Temp"] >= 70) &
                                            (weatherpy_results["Wind Speed"] <= 10) &
                                            (weatherpy_results["Cloudiness"] == 0)]

weather_criteria_df.reset_index(inplace = True)

weather_criteria_df = weather_criteria_df.drop(columns=["index", "Unnamed: 0"])

weather_criteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,natal,-5.7950,-35.2094,75.85,100,0,3.44,BR,1648189156
1,cabo rojo,18.0866,-67.1457,75.67,82,0,7.70,PR,1648189543


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,natal,-5.7950,-35.2094,75.85,100,0,3.44,BR,1648189156,
1,cabo rojo,18.0866,-67.1457,75.67,82,0,7.70,PR,1648189543,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_list = []

for index, row in hotel_df.iterrows():
    
    lat = hotel_df.loc[index]["Lat"]
    lng = hotel_df.loc[index]["Lng"]
    
    params = {"location": f"{lat},{lng}","radius": 5000,"type" : "lodging","key": g_key}
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_list.append(results[0]["name"])
        
    except:
        hotel_list.append("")
        
hotel_df["Hotel Name"] = hotel_list
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,natal,-5.7950,-35.2094,75.85,100,0,3.44,BR,1648189156,Villa Park Hotel | Natal
1,cabo rojo,18.0866,-67.1457,75.67,82,0,7.70,PR,1648189543,Hotel Perichi's


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))